In [ ]:
// Import the required modules
#r "nuget: Plotly.NET.Interactive, 3.0.2"
#r "nuget: FsODE"

// ... and open the required modules
open FsODE
open Plotly.NET

Installed Packages FsODE, 0.0.1 Plotly.NET.Interactive, 3.0.2

Loading extensions from `Plotly.NET.Interactive.dll`

In [ ]:
let modelContext = //OdeContext()
    OdeSolverMethod.RK546M //RK547M()
    |> OdeContext

## 1.1 First steps: Develop intuition for the simplest gene regulation circuits

Here, protein production occurs at rate β and degradation+dilution at rate γx. We can then write down a simple ordinary differential equation describing these dynamics

$$\quad 
\frac{dx}{dt} = \beta - (\gamma*x)
$$

### Steady State

Wird bestimmt durch die Annahme, dass die Änderung der Proteinmenge Null ist ($\frac{dx}{dt} = \beta -\gamma*x = 0$).

$$\quad 
x_{st}=\beta/\gamma
$$

In [ ]:
let beta, gamma = 
    30., // protein production rate
    50. // γ=γdilution+γdegradation

// Define a function which calculates the derivative
let dP_dt : Model = 
    fun P t ->

        let x = P[0] // Change in protein quantity

        let x' = beta - (gamma*x)

        [| x' |]

let P0 = [| 
    100.0 // x(0), protein quantity at start (x(t) -> x(0))
|]

let Ps = 
    modelContext.OdeInt(
        1., // start time point
        P0,
        dP_dt
    )
    |> SolPoints.take 10 // n timepoints
    |> SolPoints.memorize 

let proteins = Ps |> SolPoints.toPoints 1

proteins
|> Chart.Spline 
|> Chart.withTraceInfo("Protein")
|> Chart.withXAxisStyle("time")
|> Chart.withYAxisStyle("quantity")

<!-- Plotly chart will be drawn inside this DIV -->

## 1.2 Including transcription and translation as separate steps

These reactions can be described by two coupled differential equations for the mRNA (m) and protein (x):

$$\quad 
\frac{dm}{dt} = \beta_m - (\gamma_m*m)
$$

$$\quad 
\frac{dx}{dt} = \beta_x*m - (\gamma_x*x)
$$

In [ ]:
let beta_m, gamma_m = 
    2., // protein production rate
    1. // γ=γdilution+γdegradation

let beta_x, gamma_x = 
    5., // protein production rate
    4. // γ=γdilution+γdegradation

// Define a function which calculates the derivative
let dP_dt : Model = 
    fun P t ->
        let m = P[0] // Change in mRNA quantity
        let x = P[1] // Change in protein quantity

        let m' = beta_m - (gamma_m*m)
        let x' = beta_x*m - (gamma_x*x)

        [| m'; x' |]

let P0 = [| 
    0.0; // m(0), mRNA quantity at start (m(t) -> m(0))
    0.0 // x(0), protein quantity at start (x(t) -> x(0))
|]

let Ps = 
    modelContext.OdeInt(
        0., // start time point
        P0,
        dP_dt
    )
    |> SolPoints.take 15// n timepoints
    |> SolPoints.memorize 

let mRNA = Ps |> SolPoints.toPoints 1
let proteins = Ps |> SolPoints.toPoints 2

[
    mRNA
    |> Chart.Spline 
    |> Chart.withTraceInfo("mRNA");
    
    proteins
    |> Chart.Spline
    |> Chart.withTraceInfo("Proteins");

]
|> Chart.combine
|> Chart.withXAxisStyle("time")
|> Chart.withYAxisStyle("quantity")

<!-- Plotly chart will be drawn inside this DIV -->

## 1.3 From gene expression to gene regulation - adding a repressor

$$\quad
D + R \rightleftharpoons D_{occ} \quad,\quad R = Repressor,\ D = Unoccupied\ promoter,\ D_{occ} = Occupied\ promoter,\ D_{tot} = Total\ promoter
$$

Within the cell, the repressor binds and unbinds its target site. We assume that the expression level of the gene is lower when the repressor is bound and higher when it is unbound. The mean expression level of the gene is then proportional to the fraction of time that the repressor is unbound.

$$\quad
D_{tot}=D+D_{occ}
$$

We can also assume a separation of timescales between the rates of binding and unbinding of the repressor to the DNA binding site are both often fast compared to the timescales over which mRNA and protein concentrations vary. (Careful, however, in some contexts, such as mammalian cells, this is not true.)

All we need to know is the mean concentration of unoccupied binding sites, $\frac{D}{D_{tot}}.$

### Repressor binding/unbinding

$$\quad
k_+DR=k_−D_{occ} \quad,\quad k_+ = binding\ rate,\ k_− = unbinding\ rate
$$

Setzen wir nun in $D_{occ}=D_{tot}-D$ ein erhalten wir:

$$
\frac{D}{D_{tot}}=\frac{1}{1+\frac{R}{K_d}} \quad,\quad wobei\ K_d=\frac{k_−}{k_+}
$$

From this, we can write the production rate as a function of repressor concentration,

$$\quad
\beta(R) = \beta_0\frac{D}{D_{tot}}=\frac{\beta_0}{1+\frac{R}{K_d}}
$$

### Same for Activator

$$\quad
\beta(A) = \beta_0\frac{D_{occ}}{D_{tot}}=\frac{\beta_0\frac{A}{K_d}}{1+\frac{A}{K_d}}
$$

In [ ]:
let beta, gamma, K_d, R, A = 
    50., // protein production rate
    20., // γ=γdilution+γdegradation
    0.2, // repressor/activator *unbinding* rate
    1., // repressor quantity
    20. // activator quantity

// Define a function which calculates the derivative
let dP_dt : Model = 
    fun P t ->

        let x = P[0] // Change in protein quantity

        let productionRateRepressed = beta / (1.+(R/K_d))
        let productionRateActivated = (beta * (A/K_d)) / (1. + (A/K_d)) // if K_d is really low the number of A matters less, if K_d is higher the A matters more
        let productionRate = beta
        let x' = productionRateActivated - (gamma*x)

        [| x' |]

let P0 = [| 
    0.0 // x(0), protein quantity at start (x(t) -> x(0))
|]

let Ps = 
    modelContext.OdeInt(
        1., // start time point
        P0,
        dP_dt
    )
    |> SolPoints.take 10 // n timepoints
    |> SolPoints.memorize 

let proteins = Ps |> SolPoints.toPoints 1

proteins
|> Chart.Spline 
|> Chart.withTraceInfo("Protein")
|> Chart.withXAxisStyle("time")
|> Chart.withYAxisStyle("quantity")

<!-- Plotly chart will be drawn inside this DIV -->

## 1.4 Gene expression can be “leaky”

A simple way to model this is by adding an additional constant term, α0 to the expression.

$$\quad
\beta(R) = \alpha_0 + \frac{\beta_0}{1+\frac{R}{K_d}}
$$